In [1]:
# Import libraries
import pandas as pd
import geopandas as gpd
# Operations on geometries
import shapely
import numpy as np
from numpy import nan
import math
import glob 
import xarray as xr
import xarray
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
import tensorflow as tf
import statsmodels.api as sm
import xgboost
import xgboost as xgb
import csv as csv
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn import linear_model
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import TimeSeriesSplit,GridSearchCV
from sklearn.model_selection import cross_validate

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor  #Perforing grid search
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn import datasets, metrics, model_selection


import scipy
from scipy import stats
import os

import matplotlib.pyplot as plt

from scipy.stats import skew
from collections import OrderedDict

from utils_functions.utils_data import *
from utils_functions.utils_ml import *
from utils_functions.utils_RF import *
from utils_functions.utils_plot import *

/Users/noeliaotero/opt/anaconda3/envs/py39/lib/python3.9/site-packages/seaborn/apionly.py:9: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [5]:
PATH_dir = '/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/outmodels/Classical/meteo_IDdischarge/daily/'
PATH_dir_v1 = '/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/outmodels/Classical/version_HP_catch/'
PATH_data_out = '/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/outmodels/Classical/meteo_IDdischarge/daily/'
PATH_data_fout = '/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/outmodels/Classical/meteo_HPcatch/daily/'

In [6]:
list_data = readfiles(PATH_dir)
list_data_HP = readfiles(PATH_dir_v1)

In [7]:
list_data[0].columns

Index(['date', 'eic_g', 'name_p', 'ID6', 'ID', 'EZG_km2', 'lon', 'lat',
       'eic_p', 'type_g', 'discharge', 'discharge_mmd', 'hp',
       'PowerSystemResourceName', 'name_g', 'ActualGenerationOutput',
       'InstalledGenCapacity', 'capacity_p', 'generation', 't2m', 't2mmax',
       'prec', 'prec7D', 'prec15D', 'prec30D', 't2max7D', 't2max15D',
       't2max30D', 't2m7D', 't2m15D', 't2m30D', 'dis7D', 'dis15D', 'dis30D',
       'month', 'season', 'spei_1', 'spei_3', 'spei_6', 'spei_12', 'STI_1',
       'STI_2', 'STI_3', 'pred_linear', 'pred_randomforest',
       'pred_rf_Predefinesplit'],
      dtype='object')

In [8]:
import pandas as pd
from standard_precip.spi import SPI
from standard_precip.utils import plot_index

In [9]:
def get_daily_spi(mydat, vv):
    
    if (vv == 'discharge'):
        nv = ['SSID', 'SSI1', 'SSI3']
    else:
        nv = ['SPID', 'SPI1', 'SPI3']
    print(nv)
    
    spi = SPI()
    # daily
    df_spi = spi.calculate(mydat, 'date', vv, freq="D", scale=1, fit_type="lmom", dist_type="gam")
    #df_spiD = df_spi[['date','prec_calculated_index']]
    df_spiD = df_spi.drop(vv, axis=1)
    df_spiD.columns = ['date', nv[0]]
    # 1month
    df_spi_1 = spi.calculate(mydat, 'date', vv, freq="D", scale=30, fit_type="lmom", dist_type="gam")
    df_spi_1 = df_spi_1.drop(vv+'_scale_30', axis=1)
    df_spi_1.columns = ['date', nv[1]]
    # 3-month
    df_spi_3 = spi.calculate(mydat, 'date', vv, freq="D", scale=3*30, fit_type="lmom", dist_type="gam")
    df_spi_3 = df_spi_3.drop(vv+'_scale_90', axis=1)
    df_spi_3.columns = ['date', nv[2]]
    # merge all
    dfs =[df_spiD, df_spi_1, df_spi_3]
    df_all = pd.concat(dfs, join='outer', axis=1)
    df_all = df_all[['date', nv[0], nv[1], nv[2]]]
    df_all= df_all.loc[:,~df_all.columns.duplicated()]
    
    return df_all
    

In [10]:
for i in range(0, len(list_data)):
    dat = list_data[i]
    dat.date = pd.to_datetime(dat.date)
    dd = dat.copy()
    df_index = get_daily_spi(dd, 'prec')
    df_ss_index = get_daily_spi(dd, 'discharge')
    new_dat = pd.merge(dat, df_index, on='date')
    new_dat =  pd.merge(new_dat, df_ss_index, on='date')
    # save the files
    name_s = dat.name_p.unique()
    name_s = [x for x in name_s if x is not nan]
    name_s = ''.join(name_s)
    #new_dat.to_csv(PATH_data_fout  + name_s +'.csv', header=True, index=False)
    new_dat.to_csv(PATH_data_out  + name_s +'.csv', header=True, index=False)
    

['SPID', 'SPI1', 'SPI3']
['SSID', 'SSI1', 'SSI3']
['SPID', 'SPI1', 'SPI3']
['SSID', 'SSI1', 'SSI3']
['SPID', 'SPI1', 'SPI3']
['SSID', 'SSI1', 'SSI3']
['SPID', 'SPI1', 'SPI3']
['SSID', 'SSI1', 'SSI3']
['SPID', 'SPI1', 'SPI3']
['SSID', 'SSI1', 'SSI3']
['SPID', 'SPI1', 'SPI3']
['SSID', 'SSI1', 'SSI3']
